In [18]:
import os
import pandas
from datasets import load_dataset

def create_directory(subfolder):
    outdir = f"Datasets/{subfolder}"
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    return outdir


def create_csv_file_for_models(language_selection="en", dataset_split="train"):
    dataset = load_dataset("facebook/xnli", language_selection, split=dataset_split).to_pandas()

    dataset["label_name"] = "neutral"
    dataset.loc[dataset['label'] < 1, 'label_name'] = "entailment"
    dataset.loc[dataset['label'] > 1, 'label_name'] = "contradiction"

    outname = f"{dataset_split}_{language_selection}.csv"

    if dataset_split.__eq__("train"):
        bert_sample = dataset.sample(frac=0.5, random_state=42).reset_index(drop=True)
        electra_sample = dataset[~dataset.index.isin(bert_sample.index)].reset_index(drop=True)
        
        bert_sample.to_csv(os.path.join(create_directory("BERT"), outname))

        electra_sample.to_csv(os.path.join(create_directory("ELECTRA"), outname))
    else:
        dataset.to_csv(os.path.join(create_directory("Test"), outname))

In [ ]:
create_csv_file_for_models()

create_csv_file_for_models("all_languages")

create_csv_file_for_models(dataset_split="test")

create_csv_file_for_models(language_selection="all_languages", dataset_split="test")